In [1]:
''' Import the packages that we need for this tutorial '''
import sys          # sys allows us to add paths to packages
import numpy as np  # numpy 
import matplotlib.pyplot as plt

#sys.path.append('../..')  # add the pde package to the python path
sys.path.append('/Users/jankirschbaum/Documents/GitHub/D2/py-pde')
import pde